# # User Model Definition

In [4]:
# The `NubisonModel` class serves as a base class for creating custom user model classes.
# Note that modules required by UserModel must be imported within the NubisonModel class.
# This is because the UserModel is cloudpickled, and using modules imported outside of the NubisonModel class will cause errors.
from nubison_model import NubisonModel, ModelContext

class UserModel(NubisonModel):
    """A user model that extends the NubisonModel base class."""
    
    def load_model(self, context: ModelContext) -> None:
        """Load the model weights from the file.
        
        Args:
            context: Contains worker_index (0-based) for GPU initialization in parallel setups.
        """
        try:
            # Import the SimpleLinearModel class from the src directory
            # This class implements a basic linear model for text processing
            # Using absolute import path to ensure reliable imports across different execution contexts
            from src.SimpleLinearModel import SimpleLinearModel
            self.model = SimpleLinearModel("./src/weights.txt")
        except Exception as e:
            print(f"Error: {e}")

    def infer(self, x1: float, x2: float):
        """
        This method is used to return the inference result.
        """
        return {"y": self.model.calc(x1, x2)}

# # Model Register

In [5]:
# The `register` function is utilized to register the user-defined model with the system,
from nubison_model import register

# Register the user model
# The `artifact_dirs` argument specifies the folders containing the files used by the model class.
model_id = register(
    UserModel(),
    artifact_dirs="src",
    params={"desc": "This is a test model"},
    metrics={"train": 0.9, "validation": 0.8, "test": 0.7},
    tags={"inference_server_image": "ghcr.io/nubison/nubison-model:0.0.5"}
    )
print(f"Model registered: {model_id}")


2025/01/16 10:42:11 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - nubison-model (current: 0.0.6.dev0+8f73ebb.20250102231447, required: nubison-model==0.0.5)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.
2025/01/16 10:42:11 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'Default' already exists. Creating a new version of this model...
2025/01/16 10:42:11 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Default, version 8
Created version '8' of model 'Default'.
2025/01/16 10:42:11 INFO mlflow.tracking._tracking_service.client: 🏃 View run bouncy-ho

Model registered: runs:/002d6d675fbe4607b20316b5ce023128/


# # Model Testing

In [6]:
# The `test_client` function is used to test the model.
from nubison_model.Service import test_client

# Create a test client for the model.
with test_client(model_id) as client:
    # The request should include the parameters defined in the `infer` method.
    result = client.post("/infer", json={"x1": 3.1, "x2": 2})
    print(f"The result of the linear model is {result.json()['y']}.")



2025/01/16 10:42:12 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - nubison-model (current: 0.0.6.dev0+8f73ebb.20250102231447, required: nubison-model==0.0.5)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.
2025-01-16 10:42:12,123 - SimpleLinearModel - INFO - Weights loaded successfully from ./src/weights.txt.
INFO:SimpleLinearModel:Weights loaded successfully from ./src/weights.txt.
2025-01-16 10:42:12,127 - SimpleLinearModel - INFO - Calculating the result of the linear model with x1=3.1, x2=2.0.
INFO:SimpleLinearModel:Calculating the result of the linear model with x1=3.1, x2=2.0.


Prepared artifact: src -> /tmp/tmp89pftd1h/artifacts/src
The result of the linear model is 4.35.
